In [19]:
#Elasticsearch connector
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import eland as ed
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [20]:
es = Elasticsearch(['https://192.168.1.181:9200'],                  
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))

# defining the search statement to get all records in an index
searchContext = Search(using=es, index='*:so-*', doc_type='doc')
es.info()

ObjectApiResponse({'name': 'sohunter', 'cluster_name': 'sohunter', 'cluster_uuid': '586orLzYSvSqAfvBdjLTfQ', 'version': {'number': '7.16.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb', 'build_date': '2021-12-18T19:42:46.604893745Z', 'build_snapshot': False, 'lucene_version': '8.10.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [21]:
query_since_ts = 0
#body_filter = '{"query":{"bool":{"filter":[{"range":{@timestamp":{"gt":' + str(query_since_ts) + '}}}]}},"sort":[{"@timstamp":{"order":"dsc"}}]}'
body_filter = '{"query":{"@timestamp":{}},"size=10000":[{"@timestamp":{"order":"desc"}}]}'

In [22]:
#s = searchContext.query('query_string', query='message:("ts" AND "id.orig_h" AND "id.resp_h" AND "host" AND "id.resp_p" AND "method")')
 
#s = searchContext.query('query_string', query=('destination.port("*")'))

#s = searchContext.query('query_string', query='(destination.port:"*" OR http.virtual_host:"*")')

s = searchContext.query('query_string', query='http.virtual_host:*')

#s = searchContext.query('query_string', query='http.method:*')

In [23]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)


#base response query
#response = s.execute()
#if response.success():
#  df = pd.DataFrame((d.to_dict() for d in s.scan()))
#json_struct = json.loads(df.to_json(orient="records"))
#df_flat = pd.io.json.json_normalize(json_struct)
#df_flat.head(20)

#response = s.execute()
#if response.success():
    #df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})
#display(df)
#df.info()

In [24]:
response = s.execute()
df = pd.DataFrame(columns=['ts', 'source.ip','destination.ip','virtual_host','destination.port'])
if response.success():
    for d in s[:10000]:
        try:
            df = df.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'virtual_host': d['http']['virtual_host'], 'destination.port': d['destination']['port'], 'http.method': d['http']['method']}, ignore_index=True)
        except KeyError:
            pass
            
#response = s.execute()
#if response.success():
#    df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})

display(df)

df.info()

,ts,source.ip,destination.ip,virtual_host,destination.port,http.method
0,2021-11-15T23:43:44.104Z,192.168.1.154,23.45.13.176,ctldl.windowsupdate.com,80,GET
1,2021-11-15T23:41:14.617Z,192.168.1.154,23.45.13.176,ctldl.windowsupdate.com,80,GET
2,2021-11-15T23:41:14.561Z,192.168.1.154,23.45.13.176,ctldl.windowsupdate.com,80,GET
3,2021-11-15T23:01:23.400Z,192.168.1.154,23.45.13.176,ctldl.windowsupdate.com,80,GET
4,2021-11-15T23:01:23.375Z,192.168.128.25,23.45.13.176,ctldl.windowsupdate.com,80,GET
...,...,...,...,...,...,...
9995,2021-12-07T12:24:29.472Z,192.168.1.154,23.40.205.34,download.windowsupdate.com,80,GET
9996,2021-12-07T12:24:29.471Z,192.168.1.154,23.40.205.34,download.windowsupdate.com,80,GET
9997,2021-12-07T12:24:29.455Z,192.168.1.154,23.40.205.34,download.windowsupdate.com,80,GET
9998,2021-12-07T12:24:29.280Z,192.168.1.154,23.40.205.34,download.windowsupdate.com,80,GET


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                10000 non-null  object
 1   source.ip         10000 non-null  object
 2   destination.ip    10000 non-null  object
 3   virtual_host      10000 non-null  object
 4   destination.port  10000 non-null  object
 5   http.method       10000 non-null  object
dtypes: object(6)
memory usage: 468.9+ KB


In [25]:
df['ts'] = pd.to_datetime(df['ts'],infer_datetime_format=True)
df.head(5)

,ts,source.ip,destination.ip,virtual_host,destination.port,http.method
0,2021-11-15 23:43:44.104000+00:00,192.168.1.154,23.45.13.176,ctldl.windowsupdate.com,80,GET
1,2021-11-15 23:41:14.617000+00:00,192.168.1.154,23.45.13.176,ctldl.windowsupdate.com,80,GET
2,2021-11-15 23:41:14.561000+00:00,192.168.1.154,23.45.13.176,ctldl.windowsupdate.com,80,GET
3,2021-11-15 23:01:23.400000+00:00,192.168.1.154,23.45.13.176,ctldl.windowsupdate.com,80,GET
4,2021-11-15 23:01:23.375000+00:00,192.168.128.25,23.45.13.176,ctldl.windowsupdate.com,80,GET


In [26]:
# assign field/column names to variables
f_timestamp = 'ts'
f_src_ip = 'source.ip'
f_dst_ip = 'destination.ip'
f_dst_host = 'virtual_host'
f_dst_port = 'destination.port'
f_http_method = 'http.method'
f_delimiter = '\t'

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
# columns to display after the analysis
columns_to_display = ['tsScore','conn_count',f_src_ip,f_dst_ip,f_dst_host,f_http_method, f_dst_port,'deltas']

In [27]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
df = df.groupby(columns_to_groupby).agg(list)
df.head(10)

ts
source.ip     destination.ip  virtual_host                       destination.port http.method                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
192.168.1.154 104.100.128.70  tile-service.weather.microsoft.com 80               GET                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               [2021-12-04 13:04:23.973000+00:00]
              104.100.149.44  go.microsoft.com                   80               GET                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         [2021-11-15 11:35:41.768000+00:00, 2021-11-19 17:20:35.850000+00:00, 2021-11-24 15:19:45.869000+00:00, 2021-12-05 09:02:05.339000+00:00]
              104.100.156.12  crl.disa.mil                       80               GET                                                                                                                                                                                                                                                                                                               [2021-11-15 13:31:31.221000+00:00, 2021-11-15 05:31:15.028000+00:00, 2021-11-16 13:17:08.166000+00:00, 2021-11-16 13:01:19.587000+00:00, 2021-11-28 05:32:41.895000+00:00, 2021-11-29 09:34:26.246000+00:00, 2021-11-30 13:32:54.295000+00:00, 2021-11-30 07:53:55.913000+00:00, 2021-12-05 12:15:32.879000+00:00]
              104.102.249.48  download.windowsupdate.com         80               GET          [2021-11-30 02:13:03.417000+00:00, 2021-11-30 02:13:03.346000+00:00, 2021-11-30 02:13:03.159000+00:00, 2021-11-30 02:13:02.804000+00:00, 2021-11-30 02:13:02.380000+00:00, 2021-11-30 02:13:02.343000+00:00, 2021-11-30 02:13:02.012000+00:00, 2021-11-30 02:13:01.973000+00:00, 2021-11-30 02:13:01.224000+00:00, 2021-11-30 02:13:00.803000+00:00, 2021-11-30 02:13:00.589000+00:00, 2021-11-30 02:13:00.329000+00:00, 2021-11-30 02:13:00.146000+00:00, 2021-11-30 02:13:00.069000+00:00, 2021-11-30 02:12:59.616000+00:00, 2021-11-30 02:12:59.283000+00:00, 2021-11-30 02:12:58.012000+00:00, 2021-11-30 02:12:...
              104.102.249.64  ctldl.windowsupdate.com            80               GET                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [28]:
df.reset_index(inplace=True)
df.head(5)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts
0,192.168.1.154,104.100.128.70,tile-service.weather.microsoft.com,80,GET,[2021-12-04 13:04:23.973000+00:00]
1,192.168.1.154,104.100.149.44,go.microsoft.com,80,GET,"[2021-11-15 11:35:41.768000+00:00, 2021-11-19 17:20:35.850000+00:00, 2021-11-24 15:19:45.869000+00:00, 2021-12-05 09:02:05.339000+00:00]"
2,192.168.1.154,104.100.156.12,crl.disa.mil,80,GET,"[2021-11-15 13:31:31.221000+00:00, 2021-11-15 05:31:15.028000+00:00, 2021-11-16 13:17:08.166000+00:00, 2021-11-16 13:01:19.587000+00:00, 2021-11-28 05:32:41.895000+00:00, 2021-11-29 09:34:26.246000+00:00, 2021-11-30 13:32:54.295000+00:00, 2021-11-30 07:53:55.913000+00:00, 2021-12-05 12:15:32.879000+00:00]"
3,192.168.1.154,104.102.249.48,download.windowsupdate.com,80,GET,"[2021-11-30 02:13:03.417000+00:00, 2021-11-30 02:13:03.346000+00:00, 2021-11-30 02:13:03.159000+00:00, 2021-11-30 02:13:02.804000+00:00, 2021-11-30 02:13:02.380000+00:00, 2021-11-30 02:13:02.343000+00:00, 2021-11-30 02:13:02.012000+00:00, 2021-11-30 02:13:01.973000+00:00, 2021-11-30 02:13:01.224000+00:00, 2021-11-30 02:13:00.803000+00:00, 2021-11-30 02:13:00.589000+00:00, 2021-11-30 02:13:00.329000+00:00, 2021-11-30 02:13:00.146000+00:00, 2021-11-30 02:13:00.069000+00:00, 2021-11-30 02:12:59.616000+00:00, 2021-11-30 02:12:59.283000+00:00, 2021-11-30 02:12:58.012000+00:00, 2021-11-30 02:12:..."
4,192.168.1.154,104.102.249.64,ctldl.windowsupdate.com,80,GET,[2021-11-30 01:41:39.366000+00:00]


In [29]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
df['conn_count'] = df[f_timestamp].apply(lambda x: len(x))
df.head(2)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count
0,192.168.1.154,104.100.128.70,tile-service.weather.microsoft.com,80,GET,[2021-12-04 13:04:23.973000+00:00],1
1,192.168.1.154,104.100.149.44,go.microsoft.com,80,GET,"[2021-11-15 11:35:41.768000+00:00, 2021-11-19 17:20:35.850000+00:00, 2021-11-24 15:19:45.869000+00:00, 2021-12-05 09:02:05.339000+00:00]",4


In [44]:
#Filter out traffic where the connection count is quite small
df = df.loc[df['conn_count'] > 50]
df.shape

(39, 20)

In [45]:
df[f_timestamp] = df[f_timestamp].apply(lambda x: sorted(x))
df.head()

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
4,192.168.1.154,69.28.165.128,download.windowsupdate.com,80,GET,"[2021-11-16 15:19:33.560000+00:00, 2021-11-16 15:19:33.576000+00:00, 2021-11-16 15:19:33.611000+00:00, 2021-11-16 15:19:33.698000+00:00, 2021-11-16 15:19:33.736000+00:00, 2021-11-16 15:19:33.784000+00:00, 2021-11-16 15:19:33.823000+00:00, 2021-11-16 15:19:34.628000+00:00, 2021-11-16 15:19:34.727000+00:00, 2021-11-16 15:19:34.727000+00:00, 2021-11-16 15:19:34.862000+00:00, 2021-11-16 15:19:34.930000+00:00, 2021-11-16 15:19:35.765000+00:00, 2021-11-16 15:19:35.831000+00:00, 2021-11-16 15:32:18.560000+00:00, 2021-11-16 15:32:18.785000+00:00, 2021-11-16 15:32:18.940000+00:00, 2021-11-16 15:32:...",240,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 762.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 78683.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 84939.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 82602.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.0,1.0,1.000000,1.000000
23,192.168.128.25,69.28.165.128,download.windowsupdate.com,80,GET,"[2021-11-16 15:19:33.823000+00:00, 2021-11-16 15:19:33.944000+00:00, 2021-11-16 15:19:33.968000+00:00, 2021-11-16 15:19:33.993000+00:00, 2021-11-16 15:19:34.063000+00:00, 2021-11-16 15:19:34.743000+00:00, 2021-11-16 15:19:34.767000+00:00, 2021-11-16 15:19:34.886000+00:00, 2021-11-16 15:19:35.334000+00:00, 2021-11-16 15:19:35.794000+00:00, 2021-11-16 15:19:35.995000+00:00, 2021-11-19 13:23:44.414000+00:00, 2021-11-19 13:23:44.633000+00:00, 2021-11-19 13:23:45.225000+00:00, 2021-11-19 13:23:45.300000+00:00, 2021-11-19 13:23:45.415000+00:00, 2021-11-19 13:23:45.547000+00:00, 2021-11-19 13:23:...",86,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 79448.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 81146.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12204.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.0,1.0,1.000000,1.000000
35,192.168.1.154,23.45.13.184,download.windowsupdate.com,80,GET,"[2021-11-22 05:27:45.252000+00:00, 2021-11-22 05:27:45.260000+00:00, 2021-11-22 05:27:45.281000+00:00, 2021-11-22 05:27:45.310000+00:00, 2021-11-22 05:27:45.416000+00:00, 2021-11-22 05:27:45.479000+00:00, 2021-11-22 05:27:45.561000+00:00, 2021-11-22 05:27:45.614000+00:00, 2021-11-22 05:27:46.145000+00:00, 2021-11-22 05:27:46.206000+00:00, 2021-11-22 05:27:46.315000+00:00, 2021-11-22 05:27:46.410000+00:00, 2021-11-22 05:27:46.433000+00:00, 2021-11-22 05:27:46.452000+00:00, 2021-11-22 05:27:46.870000+00:00, 2021-11-22 05:27:46.939000+00:00, 2021-11-22 05:27:46.954000+00:00, 2021-11-22 05:27:...",55,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 545.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,1.0,1.0,1.000000,1.000000
45,192.168.1.154,13.107.4.50,download.windowsupdate.com,80,GET,"[2021-11-20 09:38:47.966000+00:00, 2021-11-20 09:38:48.102000+00:00, 2021-11-20 09:38:48.119000+00:00, 2021-11-20 09:38:48.157000+00:00, 2021-11-20 09:38:48.258000+00:00, 2021-11-20 09:38:48.313000+00:00, 2021-11-20 09:38:48.419000+00:00, 2021-11-20 09:38:48.571000+00:00, 2021-11-20 09:38:48.571000+00:00, 2

In [46]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
df['deltas'] = df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
df.head(2)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
4,192.168.1.154,69.28.165.128,download.windowsupdate.com,80,GET,"[2021-11-16 15:19:33.560000+00:00, 2021-11-16 15:19:33.576000+00:00, 2021-11-16 15:19:33.611000+00:00, 2021-11-16 15:19:33.698000+00:00, 2021-11-16 15:19:33.736000+00:00, 2021-11-16 15:19:33.784000+00:00, 2021-11-16 15:19:33.823000+00:00, 2021-11-16 15:19:34.628000+00:00, 2021-11-16 15:19:34.727000+00:00, 2021-11-16 15:19:34.727000+00:00, 2021-11-16 15:19:34.862000+00:00, 2021-11-16 15:19:34.930000+00:00, 2021-11-16 15:19:35.765000+00:00, 2021-11-16 15:19:35.831000+00:00, 2021-11-16 15:32:18.560000+00:00, 2021-11-16 15:32:18.785000+00:00, 2021-11-16 15:32:18.940000+00:00, 2021-11-16 15:32:...",240,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 762.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 78683.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 84939.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 82602.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.0,1.0,1.0,1.0
23,192.168.128.25,69.28.165.128,download.windowsupdate.com,80,GET,"[2021-11-16 15:19:33.823000+00:00, 2021-11-16 15:19:33.944000+00:00, 2021-11-16 15:19:33.968000+00:00, 2021-11-16 15:19:33.993000+00:00, 2021-11-16 15:19:34.063000+00:00, 2021-11-16 15:19:34.743000+00:00, 2021-11-16 15:19:34.767000+00:00, 2021-11-16 15:19:34.886000+00:00, 2021-11-16 15:19:35.334000+00:00, 2021-11-16 15:19:35.794000+00:00, 2021-11-16 15:19:35.995000+00:00, 2021-11-19 13:23:44.414000+00:00, 2021-11-19 13:23:44.633000+00:00, 2021-11-19 13:23:45.225000+00:00, 2021-11-19 13:23:45.300000+00:00, 2021-11-19 13:23:45.415000+00:00, 2021-11-19 13:23:45.547000+00:00, 2021-11-19 13:23:...",86,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 79448.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 81146.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12204.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.0,1.0,1.0,1.0


In [47]:
df['tsLow'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
df['tsMid'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
df['tsHigh'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
df['tsBowleyNum'] = df['tsLow'] + df['tsHigh'] - 2*df['tsMid']
df['tsBowleyDen'] = df['tsHigh'] - df['tsLow']
df['tsSkew'] = df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
df['tsMadm'] = df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
df['tsConnDiv'] = df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 10)

In [48]:
df.head(3)

,source.ip,destination.ip,virtual_host,destination.port,http.method,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
4,192.168.1.154,69.28.165.128,download.windowsupdate.com,80,GET,"[2021-11-16 15:19:33.560000+00:00, 2021-11-16 15:19:33.576000+00:00, 2021-11-16 15:19:33.611000+00:00, 2021-11-16 15:19:33.698000+00:00, 2021-11-16 15:19:33.736000+00:00, 2021-11-16 15:19:33.784000+00:00, 2021-11-16 15:19:33.823000+00:00, 2021-11-16 15:19:34.628000+00:00, 2021-11-16 15:19:34.727000+00:00, 2021-11-16 15:19:34.727000+00:00, 2021-11-16 15:19:34.862000+00:00, 2021-11-16 15:19:34.930000+00:00, 2021-11-16 15:19:35.765000+00:00, 2021-11-16 15:19:35.831000+00:00, 2021-11-16 15:32:18.560000+00:00, 2021-11-16 15:32:18.785000+00:00, 2021-11-16 15:32:18.940000+00:00, 2021-11-16 15:32:...",240,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 762.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 78683.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 84939.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 82602.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.0,1.0,1.0,1.0
23,192.168.128.25,69.28.165.128,download.windowsupdate.com,80,GET,"[2021-11-16 15:19:33.823000+00:00, 2021-11-16 15:19:33.944000+00:00, 2021-11-16 15:19:33.968000+00:00, 2021-11-16 15:19:33.993000+00:00, 2021-11-16 15:19:34.063000+00:00, 2021-11-16 15:19:34.743000+00:00, 2021-11-16 15:19:34.767000+00:00, 2021-11-16 15:19:34.886000+00:00, 2021-11-16 15:19:35.334000+00:00, 2021-11-16 15:19:35.794000+00:00, 2021-11-16 15:19:35.995000+00:00, 2021-11-19 13:23:44.414000+00:00, 2021-11-19 13:23:44.633000+00:00, 2021-11-19 13:23:45.225000+00:00, 2021-11-19 13:23:45.300000+00:00, 2021-11-19 13:23:45.415000+00:00, 2021-11-19 13:23:45.547000+00:00, 2021-11-19 13:23:...",86,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 79448.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 81146.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12204.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.0,1.0,1.0,1.0
35,192.168.1.154,23.45.13.184,download.windowsupdate.com,80,GET,"[2021-11-22 05:27:45.252000+00:00, 2021-11-22 05:27:45.260000+00:00, 2021-11-22 05:27:45.281000+00:00, 2021-11-22 05:27:45.310000+00:00, 2021-11-22 05:27:45.416000+00:00, 2021-11-22 05:27:45.479000+00:00, 2021-11-22 05:27:45.561000+00:00, 2021-11-22 05:27:45.614000+00:00, 2021-11-22 05:27:46.145000+00:00, 2021-11-22 05:27:46.206000+00:00, 2021-11-22 05:27:46.315000+00:00, 2021-11-22 05:27:46.410000+00:00, 2021-11-22 05:27:46.433000+00:00, 2021-11-22 05:27:46.452000+00:00, 2021-11-22 05:27:46.870000+00:00, 2021-11-22 05:27:46.939000+00:00, 2021-11-22 05:27:46.954000+00:00, 2021-11-22 05:27:...",55,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 545.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,1.0,1.0,1.0,1.0


In [49]:
df['tsSkewScore'] = 1.0 - abs(df['tsSkew'])
df['tsMadmScore'] = 1.0 - df['tsMadm']/30.0
df['tsMadmScore'] = df['tsMadmScore'].apply(lambda x: 0 if x < 0 else x)
df['tsConnCountScore'] = (df['conn_count']) / df['tsConnDiv']
df['tsConnCountScore'] = df['tsConnCountScore'].apply(lambda x: 1.0 if x > 1.0 else x)
df['tsScore'] = (((df['tsSkewScore'] + df['tsMadmScore'] + df['tsConnCountScore']) / 3.0) * 1000) / 1000
df.sort_values(by= 'tsScore', ascending=False, inplace=True, ignore_index=True)
df[columns_to_display].head(30)

,tsScore,conn_count,source.ip,destination.ip,virtual_host,http.method,destination.port,deltas
0,1.000000,240,192.168.1.154,69.28.165.128,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 762.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 78683.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 84939.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 82602.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
1,1.000000,86,192.168.128.25,69.28.165.128,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 79448.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 81146.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12204.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,1.000000,55,192.168.1.154,23.45.13.184,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 545.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0.785820,484,192.168.1.154,13.107.4.50,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3072.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 83313.0, 0.0, 0.0, 0.0, 9055.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1344.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1633.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
4,0.716795,633,192.168.1.154,209.197.3.8,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4364.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14502.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11744.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8718.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 45498.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1948.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
5,0.712585,59,192.168.128.15,208.111.176.0,download.windowsupdate.com,GET,80,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4259.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.709229,351,192.168.1.154,23.66.101.177,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 32458.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3046.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22860.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 86328.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2221.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...]"
7,0.696117,269,192.168.1.154,104.84.231.160,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3877.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0

In [50]:
df.loc[df['tsScore'] > 0.80, columns_to_display]

,tsScore,conn_count,source.ip,destination.ip,virtual_host,http.method,destination.port,deltas
0,1.0,240,192.168.1.154,69.28.165.128,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 762.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 78683.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 84939.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 82602.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
1,1.0,86,192.168.128.25,69.28.165.128,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 79448.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 81146.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12204.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,1.0,55,192.168.1.154,23.45.13.184,download.windowsupdate.com,GET,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 545.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
